In [1]:
import pandas as pd
import numpy as np
import os
from fuzzywuzzy import fuzz, process
from typing import List
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3, venn3_circles
from pandas_profiling import ProfileReport
import warnings
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
import networkx as nx
from bokeh.models import *
from bokeh.io import output_file, show
from bokeh.plotting import *
from bokeh.layouts import layout, column
from bokeh.events import ButtonClick
from platform import python_version

warnings.filterwarnings('ignore')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#read drug exposure data from your OMOP database, table drug_exposure

drug_exposures = pd.read_csv('/YOUR DIR/drug_exposure.csv', sep=',')

In [3]:
#function that reads ATC-RxNorm Relationship information from the OMOP database, tables concept and concept_relationship

def read_atc_rxnorm_relationship():
    
    # read all relationship entries for ATC to RxNorm with relationship type ATC - RxNorm pr lat, ATC - RxNorm pr up, ATC - RxNorm sec lat, ATC - RxNorm sec up   
    atc_rxnorm_relationship = pd.read_csv('/YOUR DIR/ATC-RxNorm-Rel.csv', sep='\t')
    
    return atc_rxnorm_relationship

#function to merge relationship information for each ATC code into a single row, adding now columns that contain information for the 4 mapping relationships in scope

def prepare_atc_rxorm_relationship(atc_rxnorm_relationship):
   
    # group all concept_ids based on relationship type, 
    atc_rxnorm_relationship_grouped = atc_rxnorm_relationship.groupby(['concept_id','concept_code', 'concept_name', 'relationship_id']).size().reset_index(name='count')
    
    # unmelt existing dataframe, that creates columns for each of the 4 relationship types and adds a value that indicates number of relationship for each ATC code, result is only one row per ATC code
    atc_rxnorm_relationship_grouped_unmelted=atc_rxnorm_relationship_grouped.pivot(index=['concept_id','concept_code', 'concept_name'], columns='relationship_id')
    
    # reset index, create dataframe again
    atc_rxnorm_relationship_grouped_unmelted = atc_rxnorm_relationship_grouped_unmelted['count'].reset_index()
    atc_rxnorm_relationship_grouped_unmelted.columns.name = None
    
    #change datatyp of the 4 relationship type columns to int64
    atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr lat'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr lat'].astype('Int64')
    atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr up'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm pr up'].astype('Int64')
    atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec lat'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec lat'].astype('Int64')
    atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec up'] = atc_rxnorm_relationship_grouped_unmelted['ATC - RxNorm sec up'].astype('Int64')
    atc_rxnorm_relationship_grouped_unmelted['Maps to'] = atc_rxnorm_relationship_grouped_unmelted['Maps to'].astype('Int64')
    
    
    prepared_atc_rxnorm_relationship = atc_rxnorm_relationship_grouped_unmelted
    
    return prepared_atc_rxnorm_relationship

#function to create a dataframe that only contain ATC codes with single ingredient and exactly one relationship of type ATC - RxNorm pr lat

def atc_rxnorm_single_relationship_ingredients(prepared_atc_rxnorm_relationship):
    
    atc_rxnorm_single_pr_lat_relationship = prepared_atc_rxnorm_relationship.loc[
        (prepared_atc_rxnorm_relationship['ATC - RxNorm pr lat']==1) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm pr up'].isnull()) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm sec lat'].isnull()) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm sec up'].isnull())
    ]
    
    return atc_rxnorm_single_pr_lat_relationship

#function to create a dataframe that only contain ATC codes with two ingredient and exactly two relationship of type ATC - RxNorm pr lat

def atc_rxnorm_double_relationship_ingredients(prepared_atc_rxnorm_relationship):
    
    atc_rxnorm_single_pr_lat_relationship = prepared_atc_rxnorm_relationship.loc[
        (prepared_atc_rxnorm_relationship['ATC - RxNorm pr lat']==2) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm pr up'].isnull()) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm sec lat'].isnull()) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm sec up'].isnull())
    ]
    
    return atc_rxnorm_single_pr_lat_relationship

#function to create a dataframe that only contain ATC codes with two ingredient and exactly two relationship of type ATC - RxNorm pr lat and sec lat (first and secondary ingredient)

def atc_rxnorm_single_prim_and_sec_relationship_ingredients(prepared_atc_rxnorm_relationship):
    
    atc_rxnorm_single_pr_lat_relationship = prepared_atc_rxnorm_relationship.loc[
        (prepared_atc_rxnorm_relationship['ATC - RxNorm pr up'].isnull()) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm pr lat']==1) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm sec lat']==1) & 
        (prepared_atc_rxnorm_relationship['ATC - RxNorm sec up'].isnull())
    ]
    
    return atc_rxnorm_single_pr_lat_relationship

In [4]:
#read ATC to RxNorm relationships
atc_rxnorm_relationships = read_atc_rxnorm_relationship()

#modify them to have one row per ATC code only
prepared_atc_rxnorm_relationship = prepare_atc_rxorm_relationship(atc_rxnorm_relationships)



In [5]:
#group the relationships based on the combinations in numbers for the different relationship types

prepared_atc_rxnorm_relationship_grouped = prepared_atc_rxnorm_relationship.groupby(["ATC - RxNorm pr lat","ATC - RxNorm pr up","ATC - RxNorm sec lat","ATC - RxNorm sec up","Maps to"],dropna=False).size().to_frame('size')

#export the grouped relationships

prepared_atc_rxnorm_relationship_grouped.to_csv('/YOUR DIR/prepared_atc_rxnorm_relationship_grouped.csv')

In [6]:
#print ("ATC - RxNorm pr lat ==1 but maps to == 0: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['Maps to'].isnull()) & (prepared_atc_rxnorm_relationship['ATC - RxNorm pr lat'] == 1)]))
#print ("ATC - RxNorm pr lat ==0 but maps to == 1: ",len(prepared_atc_rxnorm_relationship.loc[(prepared_atc_rxnorm_relationship['ATC - RxNorm pr lat'].isnull()) & (prepared_atc_rxnorm_relationship['Maps to'] == 1)]))

In [7]:
#merge drug exposure data (real world data, medication orders) and the prepared ATC to RxNorm Relationship information
#the result is the input data for the visualization

drug_exposures_merged_rxnorm_relationships = pd.merge(drug_exposures, prepared_atc_rxnorm_relationship, left_on='drug_concept_id', right_on='concept_id', how="left", indicator=True)

#limit drug_exposures to those that have a valid drug_concept_id that is not 0, remove all others 
drug_exposures_merged_rxnorm_relationships_valid_concepts_ATC_only = drug_exposures_merged_rxnorm_relationships.loc[drug_exposures_merged_rxnorm_relationships['drug_concept_id']!=0]

#limit merged drug_exposures to those that have been in both merged data frames
drug_exposures_with_any_rxnorm_only = drug_exposures_merged_rxnorm_relationships_valid_concepts_ATC_only.loc[drug_exposures_merged_rxnorm_relationships_valid_concepts_ATC_only['_merge']=='both']

In [8]:
#group drug exposure data and relationships and count frequency for ATC codes in a new column named "count"
drug_expsoures_with_rxnorm_grouped = drug_exposures_with_any_rxnorm_only.groupby(['drug_concept_id','concept_code','concept_name', 'ATC - RxNorm pr lat','ATC - RxNorm pr up','ATC - RxNorm sec lat','ATC - RxNorm sec up','Maps to'],dropna=False).size().reset_index(name='count').sort_values('count', ascending=False)

#fill all entries with NA with 0
drug_expsoures_with_rxnorm_grouped = drug_expsoures_with_rxnorm_grouped.fillna(0)

#add new column named "color" with hex color code, depending on ATC - RxNorm pr lat value
drug_expsoures_with_rxnorm_grouped.loc[drug_expsoures_with_rxnorm_grouped["ATC - RxNorm pr lat"] == 1, 'color'] = "#6786f6"
drug_expsoures_with_rxnorm_grouped.loc[drug_expsoures_with_rxnorm_grouped["ATC - RxNorm pr lat"] != 1, 'color'] = "#da5543"

#add new column named "exactly" and add information depending on ATC - RxNorm pr lat value
drug_expsoures_with_rxnorm_grouped.loc[drug_expsoures_with_rxnorm_grouped["ATC - RxNorm pr lat"] == 1, 'exactly'] = "exactly 1 ATC to RxNorm pr lat mapping"
drug_expsoures_with_rxnorm_grouped.loc[drug_expsoures_with_rxnorm_grouped["ATC - RxNorm pr lat"] != 1, 'exactly'] = "NOT exactly 1 ATC to RxNorm pr lat mapping"

#add new column named "sum_map" in that all relationships per ATC code are added up
drug_expsoures_with_rxnorm_grouped['sum_map'] = drug_expsoures_with_rxnorm_grouped['ATC - RxNorm pr lat']+drug_expsoures_with_rxnorm_grouped['ATC - RxNorm pr up']+drug_expsoures_with_rxnorm_grouped['ATC - RxNorm sec lat']+drug_expsoures_with_rxnorm_grouped['ATC - RxNorm sec up']

In [9]:
#calculate number of ATC codes in our drug_exposures
len(drug_expsoures_with_rxnorm_grouped)

660

In [10]:
#create new helper dataframe for the visalization

x = drug_expsoures_with_rxnorm_grouped['count']
y = drug_expsoures_with_rxnorm_grouped['sum_map']
ATC_code = drug_expsoures_with_rxnorm_grouped['concept_code']
ATC_name = drug_expsoures_with_rxnorm_grouped['concept_name']
color = drug_expsoures_with_rxnorm_grouped['color']
exactly = drug_expsoures_with_rxnorm_grouped['exactly']
ATC_RxNorm_pr_lat = drug_expsoures_with_rxnorm_grouped['ATC - RxNorm pr lat']
ATC_RxNorm_pr_up = drug_expsoures_with_rxnorm_grouped['ATC - RxNorm pr up']
ATC_RxNorm_sec_lat = drug_expsoures_with_rxnorm_grouped['ATC - RxNorm sec lat']
ATC_RxNorm_sec_up = drug_expsoures_with_rxnorm_grouped['ATC - RxNorm sec up']
Maps_to = drug_expsoures_with_rxnorm_grouped['Maps to']

df = pd.DataFrame({'ATC_code': ATC_code, 'ATC_name': ATC_name, 'frequency': x, 'TOTAL_RxNorm_mappings': y, 'ATC_RxNorm_pr_lat': ATC_RxNorm_pr_lat, 'ATC_RxNorm_pr_up': ATC_RxNorm_pr_up, 'ATC_RxNorm_sec_lat': ATC_RxNorm_sec_lat, 'ATC_RxNorm_sec_up': ATC_RxNorm_sec_up, 'Maps_to': Maps_to, 'color': color, 'exactly': exactly})


In [11]:
df

,ATC_code,ATC_name,frequency,TOTAL_RxNorm_mappings,ATC_RxNorm_pr_lat,ATC_RxNorm_pr_up,ATC_RxNorm_sec_lat,ATC_RxNorm_sec_up,Maps_to,color,exactly
509,N02BB02,"metamizole sodium; systemic, rectal",80866,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping
157,B05BB01,electrolytes; parenteral,74314,349,0,30,0,319,0,#da5543,NOT exactly 1 ATC to RxNorm pr lat mapping
27,A02BC02,pantoprazole; systemic,65861,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping
500,N02AA05,oxycodone; systemic,46434,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping
128,B01AB05,enoxaparin; parenteral,31241,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping
...,...,...,...,...,...,...,...,...,...,...,...
44,C01CA07,dobutamine; parenteral,1,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping
43,C01CA04,dopamine; parenteral,1,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping
451,L03AA02,filgrastim; parenteral,1,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping
627,S01KA02,hypromellose; ophthalmic,1,1,1,0,0,0,1,#6786f6,exactly 1 ATC to RxNorm pr lat mapping


In [12]:
#output_file("logscatter.html", title="Medication orders with ATC L5 codes and number of RxNorm relationships")
plot_source = ColumnDataSource(df)

sourcetext = ColumnDataSource(data={'ATC_code': [], 'ATC_name': [], 'frequency': [], 'TOTAL_RxNorm_mappings': [], 'ATC_RxNorm_pr_lat': [], 'ATC_RxNorm_pr_up': [], 'ATC_RxNorm_sec_lat': [], 'ATC_RxNorm_sec_up': [],'Maps_to': [], 'color': [], 'exactly': []})
  

#callback function for clean all highlighted data points
clearbutton = Button(label="Clear highlighted codes", button_type="primary", width=50)
clearbuttoncallback = CustomJS(args=dict(sourcetext=sourcetext), code="""
    var d2 = sourcetext.data;
            d2['ATC_code']=[]
            d2['ATC_name']=[]
            d2['frequency']=[]
            d2['TOTAL_RxNorm_mappings']=[]
            d2['ATC_RxNorm_pr_lat']=[]
            d2['ATC_RxNorm_pr_up']=[]
            d2['ATC_RxNorm_sec_lat']=[]
            d2['ATC_RxNorm_sec_up']=[]
            d2['Maps_to']=[]
            d2['color']=[]
            d2['exactly']=[]
    sourcetext.data = d2;        
    sourcetext.change.emit();""")
clearbutton.js_on_event(ButtonClick, clearbuttoncallback)

# Create a TextInput widget for searching the data table
search = TextInput(value = "", placeholder = "A1...", width=350)


# Create the slider
#slider = Slider(start=0, end=1100, value=0, step=1, title="Limit TOTAL ATC - RxNorm mappings to show:")


# Define a callback function to update the data based on the search string
callback_search = CustomJS(args=dict(source=plot_source, sourcetext=sourcetext), code="""
    var label = cb_obj.value.toLowerCase();
    var d1 = source.data;
    var d2 = sourcetext.data;
    for (var i = 0; i < d1['ATC_code'].length; i++) {
        if (d1['ATC_code'][i].toLowerCase().includes(label)|d1['ATC_name'][i].toLowerCase().includes(label)) {
            d2['ATC_code'].push(d1['ATC_code'][i]);
            d2['ATC_name'].push(d1['ATC_name'][i]);
            d2['frequency'].push(d1['frequency'][i]);
            d2['TOTAL_RxNorm_mappings'].push(d1['TOTAL_RxNorm_mappings'][i]);
            d2['ATC_RxNorm_pr_lat'].push(d1['ATC_RxNorm_pr_lat'][i]);
            d2['ATC_RxNorm_pr_up'].push(d1['ATC_RxNorm_pr_up'][i]);
            d2['ATC_RxNorm_sec_lat'].push(d1['ATC_RxNorm_sec_lat'][i]);
            d2['ATC_RxNorm_sec_up'].push(d1['ATC_RxNorm_sec_up'][i]);
            d2['Maps_to'].push(d1['Maps_to'][i]);
            d2['color'].push(d1['color'][i]);
            d2['exactly'].push(d1['exactly'][i]);
        }
    }
    sourcetext.data = d2;
    sourcetext.change.emit();
""")

# Attach the callback to the TextInput widget
search.js_on_change('value', callback_search) 

log_range_y = Range1d(start=1, end=10**4, name="log")
log_range_x = Range1d(start=1, end=10**5, name="log")

fig = figure(tools="pan,wheel_zoom,box_zoom,reset,save, tap, lasso_select",
       y_axis_type="log", y_range=log_range_y, x_axis_type="log", x_range=log_range_x, plot_width=1000, plot_height=1000, x_axis_label='medication order frequency', y_axis_label='Total ATC to RxNorm mappings')


fig.add_layout(Legend())

r1 = fig.circle(x='frequency', y='TOTAL_RxNorm_mappings', color='lightgrey', size=9,alpha=0.6, source=plot_source)
r2 = fig.circle(x='frequency', y='TOTAL_RxNorm_mappings',  color='color', legend='exactly', size=9, source=sourcetext)

fig.legend.location = "top_left"
fig.xaxis.axis_label_text_font_size = "14pt"
fig.yaxis.axis_label_text_font_size = "14pt"
fig.title.text_font_size = '14pt'
fig.yaxis.major_label_text_font_size = "14pt"
fig.xaxis.major_label_text_font_size = "14pt"

hover = HoverTool(tooltips=[("ATC code", "@ATC_code"),("ATC name", "@ATC_name"),("frequency", "@frequency"), ("TOTAL RxNorm mappings", "@TOTAL_RxNorm_mappings"), ("RxNorm pr lat mappings", "@ATC_RxNorm_pr_lat"), ("RxNorm pr up mappings", "@ATC_RxNorm_pr_up"), ("RxNorm sec lat mappings", "@ATC_RxNorm_sec_lat"), ("RxNorm sec up mappings", "@ATC_RxNorm_sec_up"), ("Maps to", "@Maps_to")])
hover.renderers =[r1]

fig.add_tools(hover)

columns = [
    TableColumn(field="ATC_code", title="ATC_code",
                formatter=StringFormatter(font_style="bold")),
    TableColumn(field="ATC_name", title="ATC_name"),
    TableColumn(field="frequency", title="frequency",
                editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0")),
    TableColumn(field="TOTAL_RxNorm_mappings", title="TOTAL_RxNorm_mappings", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0")),
    TableColumn(field="ATC_RxNorm_pr_lat", title="ATC_RxNorm_pr_lat", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0")),
    TableColumn(field="Maps_to", title="Maps_to", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0"))
    ]

data_table = DataTable(source=sourcetext, columns=columns, editable=False, sortable=True, selectable="checkbox", width=700)
div_header = Div(text="<br>Distribution of medication orders as ATC code by frequency and number of total ATC to RxNorm ingredient mappings<br><br>", style={'font-size': '150%', 'color': 'steelblue'}, width=1200)
div_table = Div(text="<br>Characteristics of highlighted ATC codes", style={'font-size': '100%', 'color': 'black'})
div_search = Div(text="<br>Search for ATC codes or ingredient names:", style={'font-size': '100%', 'color': 'black'})
div_blank = Div(text="")

search_col = column(div_search, search)
find_tools = column(search_col, clearbutton,)
table_col = column(div_table, data_table)

l=layout([
    [div_header],
    [find_tools, table_col],
    [fig],
    [Div(text="<br><br><br>")]])
show(l)
